**Import Libraries**


In [ ]:
import os

# List files in the current directory
print(os.listdir('.'))

In [ ]:
from google.colab import files
files.upload()



In [ ]:
from google.colab import files
files.upload()


In [ ]:
import os
os.listdir()


['.config', 'fear_greed_index.csv', 'historical_data.csv', 'sample_data']

**Load Datasets**

In [ ]:
import pandas as pd

fear_greed = pd.read_csv("fear_greed_index.csv")
trades = pd.read_csv("historical_data.csv")

print("Fear & Greed data:")
display(fear_greed.head())

print("Trades data:")
display(trades.head())


**Data Cleaning**

In [ ]:
fear_greed['date'] = pd.to_datetime(fear_greed['date'], errors='coerce')

trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], dayfirst=True, errors='coerce')

trades['date'] = trades['Timestamp IST'].dt.date
trades['date'] = pd.to_datetime(trades['date'])

display(fear_greed[['date','classification']].head())
display(trades[['Timestamp IST','date','Size USD','Closed PnL']].head())

print(trades['Timestamp IST'].isna().sum())



**Merge Datasets**

In [ ]:
df = pd.merge(
    trades,
    fear_greed[['date','classification']],
    on='date',
    how='inner'
)

print(df.shape)
display(df.head())


**Create Daily Summary**

In [ ]:
daily = df.groupby(['date','classification']).agg(
    total_volume=('Size USD','sum'),
    total_pnl=('Closed PnL','sum'),
    trade_count=('Account','count')
).reset_index()

print(daily.shape)
display(daily.head())


**Sentiment Comparison**

In [ ]:
summary = daily.groupby('classification').agg(
    avg_volume=('total_volume','mean'),
    avg_pnl=('total_pnl','mean'),
    avg_trades=('trade_count','mean')
).sort_values(by='avg_volume', ascending=False)

summary


**Visualizations**

In [ ]:
import matplotlib.pyplot as plt

summary['avg_volume'].plot(kind='bar', figsize=(8,5), title="Average Trading Volume by Market Sentiment")
plt.ylabel("Average Volume (USD)")
plt.show()


In [ ]:
summary['avg_pnl'].plot(kind='bar', figsize=(8,5), title="Average PnL by Market Sentiment")
plt.ylabel("Average Profit / Loss")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
summary['avg_volume'].plot(kind='bar', title="Average Trading Volume by Market Sentiment")
plt.ylabel("Average Volume (USD)")
plt.tight_layout()
plt.savefig("avg_volume_by_sentiment.png")
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
summary['avg_pnl'].plot(kind='bar', title="Average PnL by Market Sentiment")
plt.ylabel("Average Profit / Loss")
plt.tight_layout()
plt.savefig("avg_pnl_by_sentiment.png")
plt.show()


In [ ]:
daily.to_csv("daily_sentiment_analysis.csv", index=False)
